# Chapter 3: Undersampling

## Outline



1. Introduction
2. Remove Uniformly
    1. Random Undersampling
    2. Cluster Centroid
3. Remove Noisy Observations
    1. ENN, RENN, AllKNN
    2. Tomek Links
    3. Neighborhood Cleaning Rule
    4. Instance Hardness
4. Remove observations away from the boundary
    1. Condensed Nearest Neighbors
    2. NearMiss
5. One Sided Selection
6. Combining Undersampling and oversampling
7. Model comparison
8. Conclusion
9. Question
10. Answers

## Introduction

> Two households, both alike in dignity, In fair Verona, where we lay our scene, From ancient grudge break to new mutiny, Where civil blood makes civil hands unclean. ~ Opening lines of Romeo and Juliet, Shakespear

Imagine a town with two warring communities, viz., Montague and Capulet. They have been enemies of each other for generations. Montague is in the minority, and Capulet is in the majority in the town. Montagues are super rich and powerful. Capulets are not that well off. This creates a complex situation in the town. There are regular riots in the town because of this rivalry. One day, Montague wins the king's favor and conspires to eliminate some Capulets to bring their numbers down. The idea is that if there are fewer Capulets in the town, Montague will no longer be in the minority. King agrees to the plan as he hopes for peace after its execution. We will use this story in this chapter to illustrate various under-sampling algorithms.

Sometimes it is not sufficient to over-sample the minority class. With oversampling you can run into problems like overfitting and longer training time. To solve these problems and to approach the issue of class imbalance differently, people have thought of the opposite of oversampling, a.k.a., undersampling. 

Undersampling techniques reduce the number of samples in the majority class(es). This method has two obvious advantages over oversampling. The data size remains in check, and there is a lesser chance of overfitting. We are going to discuss the various under-sampling methods in this chapter. 

![fig 3.1](./fig-3-1.jpg)

### Fixed vs cleaning undersampling
Undersampling methods can be divided into two categories based on how many data points they remove from the majority class. These categories are Fixed methods and Cleaning methods.



In fixed methods, the number of samples in the majority class is reduced to a fixed number of samples. Primarily we reduce the number of majority class samples to the size of the minority class. For example, if there are 100 samples in the majority class and 10 samples in the minority class, after applying a fixed method, you will be left with only 10 samples of both classes. We are going to discuss three fixed methods in this chapter. They are Random Undersampling, NearMiss, and Instance Hardness-based undersampling.

In cleaning methods, the number of samples of the majority class is reduced based on some pre-determined criteria. Once this criterion is met, the algorithm doesn't care about the size of the majority or minority class. All the methods other than the three above methods belong to the cleaning methods category.

### Undersampling approaches

There are three ways the king can eliminate some Capulets. He can eliminate Capulates uniformly from the whole down, removing a few Capulets from all areas of the town. Alternatively, the king can remove Capulates who live near the houses of the Montagues. Lastly, he can remove Capulets, who live far away from the houses of the Montagues. These are the three major approaches used in under-sampling techniques. We either remove the majority samples uniformly, we remove the majority samples near the minority samples, or we remove the majority samples far from the minority samples. We can also combine the last two approaches by removing some nearby and some far-away samples. The following diagram gives the classification of these methods. 

![fig 3.2](./fig-3-2.jpg)

The figure below illustrates the difference between the two criteria.

![fig 3.3](./fig-3-3.jpg)

## Remove uniformly

### Random undersampling

The first technique the king might think of is to pick Capulets randomly and remove them from the town. This is a naïve approach. It might work, and the king might be able to bring peace to the town. But the king might cause unforeseen damage by picking up some influential Capulets. However, it is an excellent place to start our discussion. This technique is a close cousin of `Random oversampling`. In `Random undersampling`, as the name suggests, we randomly extract observations from the majority class until the classes are balanced. This technique leads to data loss, might harm the underlying structure of the data, and thus perform poorly sometimes.

Below is the code sample for using `RandomUnderSampling`.

```
from imblearn.under_sampling import RandomUnderSampler

rus = RandomUnderSampler(sampling_strategy=1.0, random_state=42)
X_res, y_res = rus.fit_resample(X, y)
```

Let's look at two diagrams before and after applying RUS.

![Fig 2.4](./fig-3-4.jpg)

## Cluster centroids

The second technique the king might follow is to divide the Capulet population into groups based on location. Then keep one Capulet from each group and remove every other Capulet. This method of undersampling is called as **ClusterCentroids** method. If there are N items in the minority class, we create N clusters from the points of the majority class. This can be done using the K-means algorithm. K-means is a clustering algorithm that groups nearby points into different clusters and assigns centroids to each group.

In ClusterCentroids technique, we first apply K-means algorithm to all of the majority class data. Then for each cluster, we keep the centroid in the data and remove all other samples. Here, the centroid might not be part of the original data.

Here is the code for using `ClusterCentroids`.
```
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=42)
X_res, y_res = cc.fit_resample(X,y)
```

![Fig 3.5](./fig-3-5.jpg)

## Strategies for removing noisy observations

The king might decide to look at the friendships and locations of the citizens before removing anyone. In business and politics, a strategy is good if its reverse is a good strategy as well. If the reverse of our strategy is not a good strategy, then there is nothing new in our strategy. We are just stating the obvious. The king might decide to remove the Capulets, who are rich and live near the Montagues. This can bring peace to the city. Let's look at some strategies to do that with our data.

### ENN, RENN, AllKNN

The king can remove Capulets based on their neighbors. If one or more of the three closest neighbors of a Capulet is Montague, the king will remove him. This technique is called **Edited Nearest Neighbors(ENN)**. **ENN** removes boundary samples to increase the separation between classes. We fit a KNN to the whole dataset. And remove the observations whose neighbors don't belong to the same class. The `imbalanced-learn` library gives us options to decide which classes we would like to resample and what kind of class arrangement the neighbors of the sample should have. 

The same options for sampling strategy are given for ENN, as discussed in Chapter 1, Introduction.

There are two different criteria we can follow for excluding the samples. Either we can choose to exclude samples whose one or more neighbors are not from the same class as themselves. Or we can decide to exclude samples whose majority of the neighbors are not from the same class as themselves.

![Fig 3.5](./fig-3-8.jpg)

Here is the code for using ENN.
```
from imblearn.under_sampling import EditedNearestNeighbours
enn = EditedNearestNeighbours(sampling_strategy='auto', n_neighbors=3, kind_sel='all')
X_res, y_res = enn.fit_resample(X, y)
```

The king might run **ENN** type of policy for few days. Each day he will calculate the nearest neighbors for the Capulets in the morning. And remove the Capulets who have one or more Montague as neighbors in the evening.

In Repeated Edited Nearest Neighbors(RENN), we repeat the process followed in ENN until there are no more samples that are removed or the maximum number of cycle count has been reached. This algorithm also removes the noisy data. It is stronger in removing the boundary samples as the algorithm is repeated several times.

You can use RENN as follows:
```
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
renn = RepeatedEditedNearestNeighbours(sampling_strategy='auto', n_neighbors=3, kind_sel='all', max_iter=200)
X_res, y_res = renn.fit_resample(X, y)
```

The king might also think of a variant of this approach. On day one, he will calculate one nearest neighbor of every Capulate. If that neighbor is Montegue, the Capulate will be removed. On day two, he will calculate two nearest neighbors of remaining Capulates. If one or more of them are Montegue, the Capulate will be removed. On day three, he will calculate the three nearest neighbors of the remaining Capulate. And remove Capulates with one or more Montegue neighbors. This is similar to **RENN** but the number of neighbors the king is calculating is increasing everyday. 

This method is called AllKNN. In this method, we repeat **ENN** but with the number of neighbors going from 1 to K. The code of AllKNN is as follows:
```
from imblearn.under_sampling import AllKNN
renn = AllKNN(sampling_strategy='auto', n_neighbors=3, kind_sel='all')
X_res, y_res = renn.fit_resample(X, y)
```

### Tomek links

In 1976, Ivan Tomek proposed the idea of Tomek links. Two points are said to form Tomek links if they belong to two different classes and there is no third point with a shorter distance to them than the distance between the two points. The intuition behind Tomek links is that “If two points are from different classes, they should not be nearest to each other.” These points are part of the noise and we can eliminate the majority member or both points to reduce noise. This is as if, the king decides to remove Capulets whose best friend is a Montague.

We can use `TomekLinks` API as follows:
```
from imblearn.under_sampling import TomekLinks
tklinks = TomekLinks(sampling_strategy='auto')
X_res, y_res = tklinks.fit_resample(X, y)
```

![Fig 3.6](./fig-3-6.jpg)

### Neighborhood cleaning rule

On top of removing Capulets whose one or more nearest neighbors are Montague. The king might decide to look at the nearest neighbors of Montagues and remove the Capulets, who might come up as one of the nearest neighbors for a Montague. In Neighborhood Cleaning Rule(NCR), we apply an ENN algorithm, train a KNN on the remaining data, and then remove all the majority class samples that are the nearest neighbors of the minority samples.  

Here is the code for using `NeighourhoodCleaningRule`.
```
from imblearn.under_sampling import NeighbourhoodCleaningRule
ncr = NeighbourhoodCleaningRule(sampling_strategy='auto', n_neighbors=3, threshold_cleaning=0.5)
X_res, y_res = ncr.fit_resample(X, y)
```

### Instance hardness

The king might ask a minister, "Which Capulets have mixed well with Montagues?" The minister based on their knowledge of the town will give a list of those Capulets. And then the king will remove the Capulets whose names are on the list. This method of using another model to identify noisy samples is known as Instance Hardness. In this method, we train a classification model on the data, like, Decision Tree, Random Forest, Linear SVM, etc. In addition to predicting the class of an instance, these classifiers can return their class probabilities. Class probabilities show the confidence the model has in classifying the instances. With Instance Hardness Threshold we remove the majority-class samples that have a low-class probability. These instances are harder to classify and are part of the noise.  

![Fig 3.7](./fig-3-7.jpg)

`Imbalance-learn` library provides an API for using `InstanceHardnessThreshold`.
```
from imblearn.under_sampling import InstanceHardnessThreshold
nm = InstanceHardnessThreshold(sampling_strategy='auto')
X_res, y_res = nm.fit_resample(X, y)
```

## Strategies for removing easy observations

The reverse of the strategy to remove the rich and famous Capulets is to remove the poor and weak Capulets. As discussed earlier, the reverse of a good strategy is also a good strategy. This section will discuss the techniques for removing the majority samples far away from the minority samples. Instead of removing the samples from the boundary between the two classes, we use them for training a model. This way, we can train a model that can better discriminate between the classes. As a downside, these models increase the noise in the data. 

### Condensed nearest neighbors

Condensed Nearest Neighbors is an algorithm that works as follows:
- We add all minority samples to a set and one randomly selected majority sample. Let's call this set C.
- We train a KNN model with k = 1.
- Now, we repeat the following four steps for each of the remaining majority samples.
    - We consider one majority sample; let's call it `e`.
    - And try to predict the class of `e` using the KNN.
    - If the predicted class matches the original class, we ignore the sample. The intuition is there is not much to learn from `e` as even a 1-NN classifier can learn it.
    - Otherwise, we add the sample to our set C and again train the 1-NN on C.

This method removes the easy-to-classify samples from the majority class.

The code to use `CondensedNearestNeighbour` looks as follows.
```
from imblearn.under_sampling import CondensedNearestNeighbour 
cnn = CondensedNearestNeighbour(random_state=42) 
X_res, y_res = cnn.fit_resample(X, y) 
```

## One sided selection

The king might decide to remove some rich and many poor Capulets. This way, only the middle-class Capulets will stay in the town. In One sided selection we do just that. This method is the combination of Condensed nearest neighbors and Tomek links. We first resample using a Condensed Nearest Neighbor. Then we remove the Tomek links from the resampled data. This technique makes a compromise between the two types of strategies discussed in the previous two sections. It reduces both noisy and easy-to-identify samples.

Here is the code for `OneSidedSelection`. When we don't provide `n_neighbors` parameter, the default value of `1` is taken.
```
from imblearn.under_sampling import OneSidedSelection 
oss = OneSidedSelection(random_state=0)
X_res, y_res = oss.fit_resample(X, y)
```

## Combining undersampling and oversampling

You might be thinking if we can combine undersampling techniques with oversampling techniques to produce even better results. 
The answer is yes and no. We can combine some strategies but not the others. Oversampling methods increase the number of samples of the minority class. But that also usually increases the noise in the data. Some undersampling techniques can help us remove the noise, e.g., ENN, Tomek links, NCR, and Instance hardness. We can combine these methods with SMOTE to produce good results. 
The combination of SMOTE with ENN and Tomek links has been heavily researched<sup>2</sup>. Also, the `imbalanced-learn` library provides support for both of them: `SMOTEENN` and `SMOTETomek`. 

[TODO]: explain more on why combining other techniques is not a good idea.

## Model Performance Comparison

Let's try to see how some popular models behave with the various undersampling techniques we went over. We use four datasets: two synthetic datasets and two real-world datasets. We compare the performance of fourteen undersampling techniques as well as no sampling on logistic regression and XGBoost models. 

You can find all the related code in the Github repo.

![model_perf_undersampling](./model_perf_undersampling.png)


## Conclusion

So, which method will work best for our data? There is no easy answer to this question. 
We need to experiment with various techniques and find the most suitable one. The key to note here is to develop an intuition about the working of these methods and have a pipeline that can help us test different techniques.

TODO: add something like what chapter-2 did, going over various topics discussed and key learnings.


## Questions

1. Explore the various undersampling APIs available from the `imbalanced-learn` library here: https://imbalanced-learn.org/stable/references/under_sampling.html
2. How does the NearMiss undersampling method work? Which class of methods does it belong to? Can you relate this method to the story we have been telling in this chapter? Please use it on the dataset in the chapter.
3. Try all the under-sampling methods discussed in this chapter on the `us_crime` dataset from UCI. You can find this dataset in the `fetch_datasets` API of the `imbalanced-learn` library. Find the undersampling method with the highest `f1-score` metric for both `LogisticRegression` and `XGBoost` models.
4. Can you identify an undersampling method of your own? (Hint: Think about combining the various approaches to undersampling in new ways.)

## Answers